## ICA Type:
FASTIca


In [1]:
%matplotlib tk 
import mne
import numpy as np
import matplotlib.pyplot as plt

In [2]:
plot_enable =1

In [3]:
epoch_file = '../preproc/Physionet_raw_epo.fif'
epochs = mne.read_epochs(epoch_file)
epochs = epochs.apply_baseline()
epochs.equalize_event_counts()

Reading ../preproc/Physionet_raw_epo.fif ...
    Found the data of interest:
        t =   -2000.00 ...    4000.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
90 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Dropped 2 epochs: 72, 85


(<EpochsFIF |  88 events (all good), -2 - 4 sec, baseline -2 – 0 sec, ~17.5 MB, data loaded,
  'T1': 44
  'T2': 44>,
 array([72, 85]))

In [4]:
epochs.plot_image()

Not setting metadata
Not setting metadata
88 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
combining channels using "gfp"


[<Figure size 640x480 with 3 Axes>]

In [6]:
tmin, tmax = -0.5, 2
epochs_data = epochs.get_data()
# labels = epochs.events[epochs.events[:,-1]!=3]
labels = epochs.events[:,-1]
labels = labels-2
# labels = epochs.events[:,-1]-2
print(f"Data Size: {epochs_data.shape}")

# train data 
epochs_train = epochs.copy().crop(tmin= tmin, tmax=tmax)
epochs_train_data = epochs_train.get_data()
print(f"Train Size: {epochs_train_data.shape}")


Data Size: (88, 27, 961)
Train Size: (88, 27, 401)


In [ ]:
## Task  Analysis
if plot_enable==1:
    task = T2
    title = str(task)[11:11+2] +' - '+ epoch_file.split('/')[-1]
    # task.plot_topomap();
    # task.plot_white(); # Noise cov required
    # task.plot_field(); # requires  surf maps
    # task.plot_sensors();
    # task.plot_topo();
    # task.plot_joint(times=[0.0, 0.2, 0.3]);#,picks=['C4','C2','C6','C1','C3','C5']);
    # task.plot_image(titles=f'{title} Image',show_names='all');
    # task.plot(proj= True, titles = '{task} - Projs - True',spatial_colors=True);
    # task.plot(proj= False, titles = '{task} -  Projs - False',spatial_colors=True);
    # task.plot(proj= 'reconstruct', titles = '{task} -  Projs - reconstruct',spatial_colors=True);
    # task.plot_topomap();
    # task.plot(gfp= "only"); # population standard deviation of the signal across channels
    ## Compare regions
    # mne.channels.combine_channels({task}, roi_dict, method='mean')
    ## Compare conditions
    evoked = dict(T1 = list(epochs['T1'].iter_evoked()), T2 = list(epochs['T2'].iter_evoked()))
    mne.viz.plot_compare_evokeds(evoked, combine='mean');
    # task_t0 = mne.combine_evoked([task, T0], weights=[1,-1])
    # task_t0.plot_joint();

In [ ]:
# Evoked data
# T0 = epochs['T0'].average() # Shape = chan x timepnts
T1 = epochs['T1'].average()
T2 = epochs['T2'].average()
print(T1.data.shape)
print(T2.data.shape)

### Classification

In [7]:
from mne.decoding import UnsupervisedSpatialFilter
from mne.preprocessing import  ICA

from sklearn.decomposition import FastICA
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.kernel_approximation import RBFSampler

In [8]:
cv = ShuffleSplit(10, test_size = 0.2, random_state=1)
cv_split = cv.split(epochs_train_data, labels)

In [ ]:
ica1 = UnsupervisedSpatialFilter(ICA(10, max_iter=200,random_state=1), average=False)
ica_data = ica1.fit_transform(epochs_train_data)


In [ ]:
ica = UnsupervisedSpatialFilter(FastICA(10, max_iter=200, tol=0.0001, w_init=None, random_state=1), average= False)
ica_data = ica.fit_transform(epochs_train_data)
# source = ica.get_sources(rawfltrd) # Estimate source(Noise) given the unmixing
# print(source.get_data().shape)
# print(ica.get_components().shape)
# print(ica.unmixing_matrix_.shape)
# print(rawfltrd_ica.get_data().shape)
info_ex = mne.create_info(10, epochs.info['sfreq'],ch_types='eeg')
# ica_evoke = mne.EvokedArray(np.mean(ica_data, axis=0), tmin=0)
ica_epoch = mne.EpochsArray(ica_data[1,:,:], info_ex, tmin=tmin)

In [17]:
ica_data.shape

(88, 10, 401)

In [ ]:
epochs.info['ch_names']

In [ ]:
lda = LDA()
svc = SVC()
dtc = DTC()
svc.fit(ica_data_T, labels)

In [ ]:
# ica = UnsupervisedSpatialFilter(FastICA(27), average = False)
rbf = RBFSampler(gamma=1, random_state=1)
clf = SVC()
pip = Pipeline([('ICA', ica),('CLF', clf)])
scores = cross_val_score(pip, epochs_train_data, labels, cv = cv, verbose=False)
scores

In [ ]:
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)
print("Classification accuracy: %f / Chance level: %f" % (np.mean(scores),
                                                          class_balance))

In [ ]:
ica_data = ica.fit_transform(epochs_data)
ev = mne.EvokedArray(np.mean(ica_data, axis=0),
                     mne.create_info(27, epochs.info['sfreq'],
                                     ch_types='eeg'))
ev.plot(show=False, window_title="ica", time_unit='s'); 

In [ ]:
sfreq = epochs.info['sfreq']
w_length = int(sfreq * 0.05)   # running classifier: window length
w_step = int(sfreq * 0.01)  # running classifier: window step size
w_start = np.arange(0, epochs_data.shape[2] - w_length, w_step)

scores_windows = []

for train_idx, test_idx in cv_split:
    y_train, y_test = labels[train_idx], labels[test_idx]

    X_train = ica.fit_transform(epochs_train_data[train_idx], y_train).reshape()
    X_test = ica.transform(epochs_train_data[test_idx])

    # fit classifier
    clf.fit(X_train, y_train)

    # running classifier: test classifier on sliding window
    score_this_window = []
    for n in w_start:
        X_test = ica.transform(epochs_data[test_idx][:, :, n:(n + w_length)])
        score_this_window.append(clf.score(X_test, y_test))
    scores_windows.append(score_this_window)
w_times = (w_start + w_length / 2.) / sfreq + epochs.tmin

plt.scatter(w_times,np.mean(scores_windows,0))

In [ ]:
sfreq = epochs.info['sfreq']
w_length = int(sfreq * 0.05)   # running classifier: window length
w_step = int(sfreq * 0.01)  # running classifier: window step size
w_start = np.arange(0, epochs_data.shape[2] - w_length, w_step)
w_start

In [ ]:
srcs = csp.transform(epochs_data)
evoked.data = np.mean(srcs,axis=0)
evoked.times = np.arange(evoked.data.shape[0])

In [ ]:
evoked.plot_topomap(times=[0, 1, 2, 3, 4]);

In [ ]:
evoked = epochs.average()
evoked.data = csp.filters_.T
evoked.times = np.arange(evoked.data.shape[0])
evoked.plot_topomap();